In [1]:
import geopandas as gpd
import pandas as pd 

In [2]:
zordi = pd.read_csv("Metro_zordi_uc_mfr_month.csv")

In [3]:
zordi

,RegionID,SizeRank,RegionName,RegionType,StateName,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,...,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31
0,102001,0,United States,country,NaN,79.0,83.0,86.0,83.0,69.0,...,77.0,81.0,84.0,84.0,77.0,71.0,63.0,56.0,48.0,46.0
1,394913,1,"New York, NY",msa,NY,55.0,51.0,68.0,99.0,96.0,...,134.0,142.0,144.0,140.0,134.0,128.0,119.0,109.0,102.0,97.0
2,753899,2,"Los Angeles, CA",msa,CA,132.0,131.0,125.0,114.0,91.0,...,95.0,96.0,99.0,102.0,95.0,89.0,81.0,72.0,61.0,59.0
3,394463,3,"Chicago, IL",msa,IL,82.0,80.0,75.0,65.0,46.0,...,149.0,150.0,149.0,145.0,132.0,121.0,106.0,94.0,84.0,83.0
4,394514,4,"Dallas, TX",msa,TX,75.0,74.0,66.0,56.0,44.0,...,42.0,43.0,44.0,41.0,34.0,30.0,28.0,26.0,22.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,395130,890,"Sterling, CO",msa,CO,NaN,NaN,NaN,NaN,NaN,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,31.0,32.0,NaN
654,394718,892,"Jamestown, ND",msa,ND,NaN,NaN,NaN,NaN,NaN,...,6.0,14.0,22.0,23.0,NaN,NaN,NaN,NaN,NaN,7.0
655,394996,915,"Portales, NM",msa,NM,NaN,NaN,NaN,NaN,NaN,...,28.0,39.0,40.0,40.0,40.0,38.0,24.0,13.0,14.0,13.0
656,786253,921,"Brownsville, TN",msa,TN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0


In [5]:
pip install requests bs4


  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.6-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.6-py3-none-any.whl (36 kB)


In [4]:
import requests, json, csv

In [8]:
pip install fake_useragent

Note: you may need to restart the kernel to use updated packages.


In [9]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import random
from fake_useragent import UserAgent
import json

In [10]:
def get_random_user_agent():
    ua = UserAgent()
    return ua.random

def scrape_zillow_listings(city, state):
    listings = []
    
    # Format URL
    base_url = f'https://www.zillow.com/{city.lower()}-{state.lower()}/rentals/apartments-condos-coops_type/'
    
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'User-Agent': get_random_user_agent(),
        'Upgrade-Insecure-Requests': '1',
    }
    
    try:
        # Add delay to be respectful
        time.sleep(random.uniform(1, 3))
        
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Look for the script tag containing the listing data
        script_tags = soup.find_all('script', type='application/json')
        for script in script_tags:
            if 'searchResults' in script.text:
                data = json.loads(script.text)
                if 'cat1' in data and 'searchResults' in data['cat1']:
                    results = data['cat1']['searchResults'].get('listResults', [])
                    
                    for result in results:
                        listing = {
                            'address': result.get('address', ''),
                            'price': result.get('price', ''),
                            'beds': result.get('beds', ''),
                            'baths': result.get('baths', ''),
                            'sqft': result.get('area', ''),
                            'url': 'https://www.zillow.com' + result.get('detailUrl', ''),
                        }
                        listings.append(listing)
        
        # Export to CSV
        if listings:
            filename = f'{city.lower()}_{state.lower()}_rentals.csv'
            with open(filename, 'w', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=listings[0].keys())
                writer.writeheader()
                writer.writerows(listings)
            print(f"Successfully exported {len(listings)} listings to {filename}")
        else:
            print("No listings found")
            
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON data: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    
    return listings

if __name__ == "__main__":
    # Example usage
    listings = scrape_zillow_listings('philadelphia', 'pa')

No listings found
